In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd

# Processing the kaggle dataset 

In [24]:
path = Path('../../../data/kaggle/closing_odds.csv')
df = pd.read_csv(path)
df.match_date = pd.to_datetime(df.match_date, format='%Y-%m-%d')
df.sort_values('match_date', inplace=True)

In [25]:
df.shape

(479440, 19)

In [26]:
df.columns

Index(['match_id', 'league', 'match_date', 'home_team', 'home_score',
       'away_team', 'away_score', 'avg_odds_home_win', 'avg_odds_draw',
       'avg_odds_away_win', 'max_odds_home_win', 'max_odds_draw',
       'max_odds_away_win', 'top_bookie_home_win', 'top_bookie_draw',
       'top_bookie_away_win', 'n_odds_home_win', 'n_odds_draw',
       'n_odds_away_win'],
      dtype='object')

In [27]:
leagues = list(df.league.unique())
buli_str = 'Germany: Bundesliga'
buli_df = df[df.league==buli_str]

In [29]:
buli_df.tail()

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,max_odds_home_win,max_odds_draw,max_odds_away_win,top_bookie_home_win,top_bookie_draw,top_bookie_away_win,n_odds_home_win,n_odds_draw,n_odds_away_win
474430,870166,Germany: Bundesliga,2015-05-23,Dortmund,3,Werder Bremen,2,1.2541,6.4234,9.9490,1.30,7.47,13.00,Interwetten,Pinnacle Sports,William Hill,29,29,29
474428,870164,Germany: Bundesliga,2015-05-23,Bayern Munich,2,Mainz,0,1.1700,7.9562,13.4545,1.20,9.48,17.00,bwin,Pinnacle Sports,Sportingbet,29,29,29
474429,870165,Germany: Bundesliga,2015-05-23,B. Monchengladbach,1,Augsburg,3,1.4703,4.5645,6.3879,1.53,5.00,7.16,Coral,Pinnacle Sports,12BET,29,29,29
475447,871556,Germany: Bundesliga,2015-05-28,Hamburger SV,1,Karlsruher,1,2.2679,3.1768,3.2704,2.46,3.40,3.85,Pinnacle Sports,myBet,myBet,28,28,28
476364,872810,Germany: Bundesliga,2015-06-01,Karlsruher,1,Hamburger SV,1,2.1682,3.3089,3.3564,2.30,3.45,3.87,BetVictor,SBOBET,Pinnacle Sports,28,28,28


In [21]:
len(buli_df)

2747

In [22]:
17*18

306

In [23]:
2747/306

8.977124183006536